In [1]:
import pandas as pd
import duckdb

In [2]:
# Load 7 tables from CSV files (update paths accordingly)
df1 = pd.read_csv("/Users/krithikreddy/Documents/Mod3/IP/data 2/Bagging activities.csv")
df2 = pd.read_csv("/Users/krithikreddy/Documents/Mod3/IP/data 2/Beginning inventory.csv")
df3 = pd.read_csv("/Users/krithikreddy/Documents/Mod3/IP/data 2/Dispo activities.csv")
df4 = pd.read_csv("/Users/krithikreddy/Documents/Mod3/IP/data 2/Location transfers.csv")
df5 = pd.read_csv("/Users/krithikreddy/Documents/Mod3/IP/data 2/Locations.csv")
df6 = pd.read_csv("/Users/krithikreddy/Documents/Mod3/IP/data 2/Returns.csv")
df7 = pd.read_csv("/Users/krithikreddy/Documents/Mod3/IP/data 2/Shipments.csv")

In [3]:
# Assign table names in a dictionary
tables = {
    "Bagging": df1,
    "Beginning": df2,
    "Dispo": df3,
    "LocationTransfers": df4,
    "Locations": df5,
    "Returns": df6,
    "Shipments": df7,
}

print("Tables loaded successfully!")

Tables loaded successfully!


In [4]:
# Convert ACTIVITYDATE in original Shipments DataFrame before using it in SQL
tables["Shipments"]["ACTIVITYDATE"] = pd.to_datetime(
    tables["Shipments"]["ACTIVITYDATE"], errors='coerce'
)

tables["LocationTransfers"]["ACTIVITYDATE"] = pd.to_datetime(
    tables["LocationTransfers"]["ACTIVITYDATE"], errors='coerce'
)

# Register the corrected tables in DuckDB
duckdb.register("Shipments", tables["Shipments"])
duckdb.register("LocationTransfers", tables["LocationTransfers"])

/var/folders/1n/088vtf8j46jdg4lqkg0qwtj00000gn/T/ipykernel_57527/878445025.py:2: UserWarning: Parsing dates in %d-%m-%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  tables["Shipments"]["ACTIVITYDATE"] = pd.to_datetime(


In [5]:
# Convert ACTIVITYDATE to DATE before running SQL (Fix in Pandas)
tables["Shipments"]["ACTIVITYDATE"] = pd.to_datetime(tables["Shipments"]["ACTIVITYDATE"], errors='coerce')

In [6]:
# Register all tables in DuckDB
for table_name, df in tables.items():
    duckdb.register(table_name, df)

In [7]:
# Create and store the transformed Shipments table
query = """
    CREATE OR REPLACE TABLE shipments_updated AS 
    SELECT 
        strftime('%Y-%m-%d', ACTIVITYDATE) AS StandardizedDate,
        *,
        strftime('%Y-%m-%d', ACTIVITYDATE) || '-' || KEYCUSTOMER || '-' || LOCATION || '-' || DLVRROUTE || '-' || DLVRSTOPNMBR 
        AS shipment_id
    FROM Shipments
"""

In [8]:
# Execute the query to create the new table
duckdb.query(query)

In [9]:
query="""
WITH Agg AS (
    SELECT
       KEYCUSTOMER,
       FISCALYEAR,
       COUNT(distinct shipment_id) AS shipment_count,
       SUM(NETUNITS) AS total_units,
       CEIL(ABS(SUM(NETUNITS)) / 675.0) AS required_shipments
    FROM shipments_updated
    GROUP BY KEYCUSTOMER, FISCALYEAR
)
SELECT
  KEYCUSTOMER AS "Customer ID",
  
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2020 THEN shipment_count END), 0) AS "Shipments in 2020",
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2020 THEN GREATEST(shipment_count - required_shipments, 0) END), 0) AS "Excess Shipments in 2020",
  
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2021 THEN shipment_count END), 0) AS "Shipments in 2021",
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2021 THEN GREATEST(shipment_count - required_shipments, 0) END), 0) AS "Excess Shipments in 2021",
  
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2022 THEN shipment_count END), 0) AS "Shipments in 2022",
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2022 THEN GREATEST(shipment_count - required_shipments, 0) END), 0) AS "Excess Shipments in 2022",
  
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2023 THEN shipment_count END), 0) AS "Shipments in 2023",
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2023 THEN GREATEST(shipment_count - required_shipments, 0) END), 0) AS "Excess Shipments in 2023",
  
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2024 THEN shipment_count END), 0) AS "Shipments in 2024",
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2024 THEN GREATEST(shipment_count - required_shipments, 0) END), 0) AS "Excess Shipments in 2024"
  
FROM Agg
GROUP BY KEYCUSTOMER
ORDER BY KEYCUSTOMER;
"""



# Execute the query
result = duckdb.query(query)

# Print the output
result

┌─────────────┬───────────────────┬──────────────────────────┬───────────────────┬──────────────────────────┬───────────────────┬──────────────────────────┬───────────────────┬──────────────────────────┬───────────────────┬──────────────────────────┐
│ Customer ID │ Shipments in 2020 │ Excess Shipments in 2020 │ Shipments in 2021 │ Excess Shipments in 2021 │ Shipments in 2022 │ Excess Shipments in 2022 │ Shipments in 2023 │ Excess Shipments in 2023 │ Shipments in 2024 │ Excess Shipments in 2024 │
│    int64    │       int64       │          double          │       int64       │          double          │       int64       │          double          │       int64       │          double          │       int64       │          double          │
├─────────────┼───────────────────┼──────────────────────────┼───────────────────┼──────────────────────────┼───────────────────┼──────────────────────────┼───────────────────┼──────────────────────────┼───────────────────┼────────────────────────

In [10]:
query="""
WITH Agg AS (
    SELECT
       KEYCUSTOMER,
       FISCALYEAR,
       COUNT(DISTINCT shipment_id) AS shipment_count,
       SUM(NETUNITS) AS total_units,
       CEIL(ABS(SUM(NETUNITS)) / 675.0) AS required_shipments
    FROM shipments_updated
    GROUP BY KEYCUSTOMER, FISCALYEAR
)
SELECT
  KEYCUSTOMER AS "Customer ID",
  
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2020 THEN shipment_count END), 0) AS "Shipments in 2020",
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2020 THEN required_shipments END), 0) AS "Minimum Shipments in 2020",
  
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2021 THEN shipment_count END), 0) AS "Shipments in 2021",
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2021 THEN required_shipments END), 0) AS "Minimum Shipments in 2021",
  
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2022 THEN shipment_count END), 0) AS "Shipments in 2022",
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2022 THEN required_shipments END), 0) AS "Minimum Shipments in 2022",
  
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2023 THEN shipment_count END), 0) AS "Shipments in 2023",
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2023 THEN required_shipments END), 0) AS "Minimum Shipments in 2023",
  
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2024 THEN shipment_count END), 0) AS "Shipments in 2024",
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2024 THEN required_shipments END), 0) AS "Minimum Shipments in 2024"
  
FROM Agg
GROUP BY KEYCUSTOMER
ORDER BY KEYCUSTOMER;
"""



# Execute the query
result = duckdb.query(query)

# Print the output
result

┌─────────────┬───────────────────┬───────────────────────────┬───────────────────┬───────────────────────────┬───────────────────┬───────────────────────────┬───────────────────┬───────────────────────────┬───────────────────┬───────────────────────────┐
│ Customer ID │ Shipments in 2020 │ Minimum Shipments in 2020 │ Shipments in 2021 │ Minimum Shipments in 2021 │ Shipments in 2022 │ Minimum Shipments in 2022 │ Shipments in 2023 │ Minimum Shipments in 2023 │ Shipments in 2024 │ Minimum Shipments in 2024 │
│    int64    │       int64       │          double           │       int64       │          double           │       int64       │          double           │       int64       │          double           │       int64       │          double           │
├─────────────┼───────────────────┼───────────────────────────┼───────────────────┼───────────────────────────┼───────────────────┼───────────────────────────┼───────────────────┼───────────────────────────┼───────────────────┼─────

In [11]:
query="""
WITH Agg AS (
    SELECT
       KEYCUSTOMER,
       FISCALYEAR,
       COUNT(DISTINCT shipment_id) AS shipment_count,
       SUM(NETUNITS) AS total_units,
       CEIL(ABS(SUM(NETUNITS)) / 675.0) AS required_shipments
    FROM shipments_updated
    GROUP BY KEYCUSTOMER, FISCALYEAR
),
Pivoted AS (
    SELECT
      KEYCUSTOMER AS "Customer ID",
      
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2020 THEN shipment_count END), 0) AS "Shipments in 2020",
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2020 THEN required_shipments END), 0) AS "Minimum Shipments in 2020",
      
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2021 THEN shipment_count END), 0) AS "Shipments in 2021",
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2021 THEN required_shipments END), 0) AS "Minimum Shipments in 2021",
      
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2022 THEN shipment_count END), 0) AS "Shipments in 2022",
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2022 THEN required_shipments END), 0) AS "Minimum Shipments in 2022",
      
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2023 THEN shipment_count END), 0) AS "Shipments in 2023",
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2023 THEN required_shipments END), 0) AS "Minimum Shipments in 2023",
      
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2024 THEN shipment_count END), 0) AS "Shipments in 2024",
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2024 THEN required_shipments END), 0) AS "Minimum Shipments in 2024"
      
    FROM Agg
    GROUP BY KEYCUSTOMER
)
SELECT *
FROM Pivoted
WHERE 
    "Shipments in 2020" < "Minimum Shipments in 2020"
 OR "Shipments in 2021" < "Minimum Shipments in 2021"
 OR "Shipments in 2022" < "Minimum Shipments in 2022"
 OR "Shipments in 2023" < "Minimum Shipments in 2023"
 OR "Shipments in 2024" < "Minimum Shipments in 2024"
ORDER BY "Customer ID";
"""



# Execute the query
result = duckdb.query(query)

# Print the output
result

┌─────────────┬───────────────────┬───────────────────────────┬───────────────────┬───────────────────────────┬───────────────────┬───────────────────────────┬───────────────────┬───────────────────────────┬───────────────────┬───────────────────────────┐
│ Customer ID │ Shipments in 2020 │ Minimum Shipments in 2020 │ Shipments in 2021 │ Minimum Shipments in 2021 │ Shipments in 2022 │ Minimum Shipments in 2022 │ Shipments in 2023 │ Minimum Shipments in 2023 │ Shipments in 2024 │ Minimum Shipments in 2024 │
│    int64    │       int64       │          double           │       int64       │          double           │       int64       │          double           │       int64       │          double           │       int64       │          double           │
├─────────────┼───────────────────┼───────────────────────────┼───────────────────┼───────────────────────────┼───────────────────┼───────────────────────────┼───────────────────┼───────────────────────────┼───────────────────┼─────

In [12]:
query="""
WITH Agg AS (
    SELECT
       KEYCUSTOMER,
       FISCALYEAR,
       COUNT(DISTINCT shipment_id) AS shipment_count,
       SUM(NETUNITS) AS total_units,
       CEIL(ABS(SUM(NETUNITS)) / 675.0) AS required_shipments
    FROM shipments_updated
    GROUP BY KEYCUSTOMER, FISCALYEAR
),
Pivoted AS (
    SELECT
      KEYCUSTOMER AS "Customer ID",
      
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2020 THEN shipment_count END), 0) AS "Shipments in 2020",
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2020 THEN required_shipments END), 0) AS "Minimum Shipments in 2020",
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2020 THEN ABS(total_units) END), 0) AS "Units in 2020",
      
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2021 THEN shipment_count END), 0) AS "Shipments in 2021",
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2021 THEN required_shipments END), 0) AS "Minimum Shipments in 2021",
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2021 THEN ABS(total_units) END), 0) AS "Units in 2021",
      
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2022 THEN shipment_count END), 0) AS "Shipments in 2022",
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2022 THEN required_shipments END), 0) AS "Minimum Shipments in 2022",
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2022 THEN ABS(total_units) END), 0) AS "Units in 2022",
      
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2023 THEN shipment_count END), 0) AS "Shipments in 2023",
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2023 THEN required_shipments END), 0) AS "Minimum Shipments in 2023",
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2023 THEN ABS(total_units) END), 0) AS "Units in 2023",
      
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2024 THEN shipment_count END), 0) AS "Shipments in 2024",
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2024 THEN required_shipments END), 0) AS "Minimum Shipments in 2024",
      COALESCE(MAX(CASE WHEN FISCALYEAR = 2024 THEN ABS(total_units) END), 0) AS "Units in 2024"
      
    FROM Agg
    GROUP BY KEYCUSTOMER
)
SELECT *
FROM Pivoted
WHERE 
    "Shipments in 2020" < "Minimum Shipments in 2020"
 OR "Shipments in 2021" < "Minimum Shipments in 2021"
 OR "Shipments in 2022" < "Minimum Shipments in 2022"
 OR "Shipments in 2023" < "Minimum Shipments in 2023"
 OR "Shipments in 2024" < "Minimum Shipments in 2024"
ORDER BY "Customer ID";
"""



# Execute the query
result = duckdb.query(query)

# Print the output
result

┌─────────────┬───────────────────┬───────────────────────────┬───────────────┬───────────────────┬───────────────────────────┬───────────────┬───────────────────┬───────────────────────────┬───────────────┬───────────────────┬───────────────────────────┬───────────────┬───────────────────┬───────────────────────────┬───────────────┐
│ Customer ID │ Shipments in 2020 │ Minimum Shipments in 2020 │ Units in 2020 │ Shipments in 2021 │ Minimum Shipments in 2021 │ Units in 2021 │ Shipments in 2022 │ Minimum Shipments in 2022 │ Units in 2022 │ Shipments in 2023 │ Minimum Shipments in 2023 │ Units in 2023 │ Shipments in 2024 │ Minimum Shipments in 2024 │ Units in 2024 │
│    int64    │       int64       │          double           │    double     │       int64       │          double           │    double     │       int64       │          double           │    double     │       int64       │          double           │    double     │       int64       │          double           │    doubl

In [13]:
import duckdb
import pandas as pd

query = """
WITH Agg AS (
    SELECT
       KEYCUSTOMER,
       FISCALYEAR,
       COUNT(DISTINCT shipment_id) AS shipment_count,
       SUM(NETUNITS) AS total_units,
       CEIL(ABS(SUM(NETUNITS)) / 675.0) AS required_shipments
    FROM shipments_updated
    GROUP BY KEYCUSTOMER, FISCALYEAR
)
SELECT
  KEYCUSTOMER AS "Customer ID",
  
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2020 THEN shipment_count END), 0) AS "Shipments in 2020",
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2020 THEN required_shipments END), 0) AS "Minimum Shipments in 2020",
  
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2021 THEN shipment_count END), 0) AS "Shipments in 2021",
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2021 THEN required_shipments END), 0) AS "Minimum Shipments in 2021",
  
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2022 THEN shipment_count END), 0) AS "Shipments in 2022",
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2022 THEN required_shipments END), 0) AS "Minimum Shipments in 2022",
  
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2023 THEN shipment_count END), 0) AS "Shipments in 2023",
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2023 THEN required_shipments END), 0) AS "Minimum Shipments in 2023",
  
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2024 THEN shipment_count END), 0) AS "Shipments in 2024",
  COALESCE(MAX(CASE WHEN FISCALYEAR = 2024 THEN required_shipments END), 0) AS "Minimum Shipments in 2024"
  
FROM Agg
GROUP BY KEYCUSTOMER
ORDER BY KEYCUSTOMER;
"""

# Execute the query and convert the result to a DataFrame
result_df = duckdb.query(query).to_df()

# Print the DataFrame
print(result_df)

# Write the DataFrame to a CSV file in a directory where you have write permissions.
# For example, using the /tmp/ directory on Unix-like systems.
result_df.to_csv("/Users/krithikreddy/Downloads/1st_output.csv", index=False)

       Customer ID  Shipments in 2020  Minimum Shipments in 2020  \
0                2                  5                        1.0   
1                4                  3                        1.0   
2                9                  1                        1.0   
3               10                  1                        1.0   
4               12                  2                        1.0   
...            ...                ...                        ...   
11165       430964                  0                        0.0   
11166       430981                  0                        0.0   
11167       431109                  0                        0.0   
11168       432296                  0                        0.0   
11169       432513                  0                        0.0   

       Shipments in 2021  Minimum Shipments in 2021  Shipments in 2022  \
0                      3                        1.0                  3   
1                      1           

In [3]:
import pandas as pd

# Load the CSV file
file_path = "/Users/krithikreddy/Documents/Mod3/IP/data 2/Shipments.csv"
df = pd.read_csv(file_path)

# Filter data for fiscal years 2020 to 2024
df_filtered = df[df["FISCALYEAR"].isin([2020, 2021, 2022, 2023, 2024])]

# Create a pivot table: 1 if activity exists in that year for that customer
activity = (
    df_filtered.groupby(["KEYCUSTOMER", "FISCALYEAR"])
    .size()
    .unstack(fill_value=0)
    .applymap(lambda x: 1 if x > 0 else 0)
)

# Ensure all years are included
for year in [2020, 2021, 2022, 2023, 2024]:
    if year not in activity.columns:
        activity[year] = 0

# Reorder the columns by year
activity = activity[[2020, 2021, 2022, 2023, 2024]]

# Rename columns
activity.columns = [f"Activity in {year}" for year in activity.columns]

# Reset index to include KEYCUSTOMER as a column
activity.reset_index(inplace=True)

# Display the final output
print(activity)


activity.to_excel("/Users/krithikreddy/Downloads/customer_activity_by_year.xlsx", index=False)


/var/folders/1n/088vtf8j46jdg4lqkg0qwtj00000gn/T/ipykernel_19415/3699007860.py:12: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_filtered.groupby(["KEYCUSTOMER", "FISCALYEAR"])


       KEYCUSTOMER  Activity in 2020  Activity in 2021  Activity in 2022  \
0                2                 1                 1                 1   
1                4                 1                 1                 1   
2                9                 1                 1                 1   
3               10                 1                 1                 1   
4               12                 1                 1                 1   
...            ...               ...               ...               ...   
11165       430964                 0                 0                 0   
11166       430981                 0                 0                 0   
11167       431109                 0                 0                 0   
11168       432296                 0                 0                 0   
11169       432513                 0                 0                 0   

       Activity in 2023  Activity in 2024  
0                     1                 1  

In [4]:
df8 = pd.read_csv("/Users/krithikreddy/Downloads/customer_activity_by_year.csv")
# Assign table names in a dictionary
tables = {
    "CustomerActivity": df8,
}

print("Tables loaded successfully!")

Tables loaded successfully!


In [5]:
# Assign table names in a dictionary
tables = {
    "CustomerActivity": df8,
}

print("Tables loaded successfully!")

Tables loaded successfully!


In [6]:
import pandas as pd
import duckdb

# Load the CSV file into a DataFrame
df8 = pd.read_csv("/Users/krithikreddy/Downloads/customer_activity_by_year.csv")
print("Tables loaded successfully!")

# Register the DataFrame as a DuckDB table called "CustomerActivity"
duckdb.register("CustomerActivity", df8)

# Define the query
query = """
WITH Activity AS (
  SELECT 
    SUM("Activity in 2020") AS active_2020,
    SUM("Activity in 2021") AS active_2021,
    SUM("Activity in 2022") AS active_2022,
    SUM("Activity in 2023") AS active_2023,
    SUM("Activity in 2024") AS active_2024,
    
    (SELECT COUNT(*) FROM CustomerActivity 
      WHERE "Activity in 2020" = 1 AND "Activity in 2021" = 1) AS retained_2021,
      
    (SELECT COUNT(*) FROM CustomerActivity 
      WHERE "Activity in 2021" = 1 AND "Activity in 2022" = 1) AS retained_2022,
      
    (SELECT COUNT(*) FROM CustomerActivity 
      WHERE "Activity in 2022" = 1 AND "Activity in 2023" = 1) AS retained_2023,
      
    (SELECT COUNT(*) FROM CustomerActivity 
      WHERE "Activity in 2023" = 1 AND "Activity in 2024" = 1) AS retained_2024,
      
    (SELECT COUNT(*) FROM CustomerActivity 
      WHERE "Activity in 2020" = 1 
        AND "Activity in 2021" = 1 
        AND "Activity in 2022" = 1 
        AND "Activity in 2023" = 1 
        AND "Activity in 2024" = 1) AS consistent_2020,
        
    (SELECT COUNT(*) FROM CustomerActivity 
      WHERE "Activity in 2021" = 1 
        AND "Activity in 2022" = 1 
        AND "Activity in 2023" = 1 
        AND "Activity in 2024" = 1) AS consistent_2021,
        
    (SELECT COUNT(*) FROM CustomerActivity 
      WHERE "Activity in 2022" = 1 
        AND "Activity in 2023" = 1 
        AND "Activity in 2024" = 1) AS consistent_2022,
        
    (SELECT COUNT(*) FROM CustomerActivity 
      WHERE "Activity in 2023" = 1 
        AND "Activity in 2024" = 1) AS consistent_2023,
        
    (SELECT COUNT(*) FROM CustomerActivity 
      WHERE "Activity in 2024" = 1) AS consistent_2024
  FROM CustomerActivity
)
SELECT 
  '2020' AS "Year",
  NULL AS "Change in customers",  -- no previous year
  NULL AS "Retention from previous year",
  consistent_2020 AS "Consistent Customers till 2024"
FROM Activity
UNION ALL
SELECT 
  '2021' AS "Year",
  ROUND((active_2021 - active_2020) * 1.0 / active_2020, 4) AS "Change in customers",
  ROUND(retained_2021 * 1.0 / active_2020, 4) AS "Retention from previous year",
  consistent_2021 AS "Consistent Customers till 2024"
FROM Activity
UNION ALL
SELECT 
  '2022' AS "Year",
  ROUND((active_2022 - active_2021) * 1.0 / active_2021, 4) AS "Change in customers",
  ROUND(retained_2022 * 1.0 / active_2021, 4) AS "Retention from previous year",
  consistent_2022 AS "Consistent Customers till 2024"
FROM Activity
UNION ALL
SELECT 
  '2023' AS "Year",
  ROUND((active_2023 - active_2022) * 1.0 / active_2021, 4) AS "Change in customers",
  ROUND(retained_2023 * 1.0 / active_2022, 4) AS "Retention from previous year",
  consistent_2023 AS "Consistent Customers till 2024"
FROM Activity
UNION ALL
SELECT 
  '2024' AS "Year",
  ROUND((active_2024 - active_2023) * 1.0 / active_2023, 4) AS "Change in customers",
  ROUND(retained_2024 * 1.0 / active_2023, 4) AS "Retention from previous year",
  consistent_2024 AS "Consistent Customers till 2024"
FROM Activity;
"""

# Execute the query and convert the result to a DataFrame
result_df = duckdb.query(query).to_df()

# Print the DataFrame
print(result_df)

# Write the DataFrame to a CSV file (ensure that the path is writable)
result_df.to_csv("/Users/krithikreddy/Downloads/2nd_output.csv", index=False)

Tables loaded successfully!
   Year  Change in customers  Retention from previous year  \
0  2020                  NaN                           NaN   
1  2021              -0.0054                        0.7152   
2  2022              -0.1289                        0.6454   
3  2023              -0.2435                        0.5865   
4  2024              -0.1514                        0.6844   

   Consistent Customers till 2024  
0                            1866  
1                            2134  
2                            2544  
3                            2925  
4                            3627  
